In [1]:
import tensornetwork as tn
import numpy as np
import matplotlib.pyplot as plt

In [44]:
import numpy as np

def initialize_state(n_qubits):
    """Initialize the quantum state to an equal superposition state."""
    state = np.ones((2,) * n_qubits) / np.sqrt(2**n_qubits)
    return state

def apply_single_qubit_gate(state, gate, target_qubit):
    """Apply a single-qubit gate to a tensor network state."""
    gate = gate.reshape((2, 2))
    axes = list(range(len(state.shape)))
    axes[target_qubit] = axes[-1]
    axes[-1] = target_qubit
    state = np.tensordot(state, gate, axes=(target_qubit, 0))
    return np.moveaxis(state, -1, target_qubit)

def apply_multi_controlled_z(state):
    """Apply a multi-controlled-Z gate to the tensor network state."""
    n_qubits = len(state.shape)
    # Multi-controlled-Z is equivalent to flipping the sign of the |11...1> state
    indices = tuple([1] * n_qubits)
    state[indices] *= -1
    return state

def apply_oracle(state, target_state):
    """Apply the oracle by flipping the phase of the target state."""
    shape = state.shape
    indices = tuple(int(b) for b in format(target_state, f'0{len(shape)}b'))
    state[indices] *= -1
    return state

def apply_diffusion(state):
    """Apply the diffusion operator."""
    n_qubits = len(state.shape)
    H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
    X = np.array([[0, 1], [1, 0]])
    Z = np.array([[1, 0], [0, -1]])

    # Apply H gate to all qubits
    for qubit in range(n_qubits):
        state = apply_single_qubit_gate(state, H, qubit)

    # Apply X gate to all qubits
    for qubit in range(n_qubits):
        state = apply_single_qubit_gate(state, X, qubit)

    # Apply multi-controlled-Z gate
    state = apply_multi_controlled_z(state)

    # Apply Z gate to 0th qubit
    state = apply_single_qubit_gate(state, Z, 0)

    # Apply X gate to all qubits
    for qubit in range(n_qubits):
        state = apply_single_qubit_gate(state, X, qubit)

    # Apply Z gate to 0th qubit
    state = apply_single_qubit_gate(state, Z, 0)

    # Apply H gate to all qubits
    for qubit in range(n_qubits):
        state = apply_single_qubit_gate(state, H, qubit)

    return state

def measure_state(state):
    """Measure the quantum state to find the most probable state."""
    probabilities = np.abs(state)**2
    most_probable_state = np.unravel_index(np.argmax(probabilities), state.shape)
    return most_probable_state

# Number of qubits
# n_qubits = 3

# # Initialize the quantum state
# state = initialize_state(n_qubits)

# # Target state to search for (in decimal representation)
# target_state = 5  # Corresponds to the binary state '101'

# # Number of Grover iterations
# num_iterations = int(np.floor(np.pi / 4 * np.sqrt(2**n_qubits)))

# # Perform Grover iterations
# for _ in range(num_iterations):
#     state = apply_oracle(state, target_state)
#     state = apply_diffusion(state)

# # Measure the final state
# result = measure_state(state)
# result_bin = ''.join(map(str, result))
# print("Measurement result: ", result_bin)


In [49]:
state = initialize_state(3)
state

array([[[0.35355339, 0.35355339],
        [0.35355339, 0.35355339]],

       [[0.35355339, 0.35355339],
        [0.35355339, 0.35355339]]])

In [50]:
state = apply_oracle(state, 5)
state

array([[[ 0.35355339,  0.35355339],
        [ 0.35355339,  0.35355339]],

       [[ 0.35355339, -0.35355339],
        [ 0.35355339,  0.35355339]]])

In [51]:
state = apply_diffusion(state)
state

array([[[0.1767767 , 0.1767767 ],
        [0.1767767 , 0.1767767 ]],

       [[0.1767767 , 0.88388348],
        [0.1767767 , 0.1767767 ]]])

In [52]:
np.argmax(state.reshape(-1))

5